In [ ]:
sys.path.insert(0, snakemake.input.data2dd)

In [ ]:
import pandas as pd
from data2dd_funcs import wrapdd

In [ ]:
files = [
    snakemake.input["areashydro"],
    snakemake.input["areassolar"],
    snakemake.input["areaswindon"],
    snakemake.input["areaswindoff"],
]

areas = pd.concat(([pd.read_csv(f, header=None, sep=" ") for f in files]))

wrapdd(areas, "area", "parameter", outfile=snakemake.output[0])

In [ ]:
areas[["tech", "zone", "region"]] = areas[0].str.split(".", expand=True)

# Add missing regions in exist_gen_r (only with nuts2 spatial option)
if snakemake.wildcards.spatial == "nuts2":
    excel_exist_pcap_r = pd.read_excel(
        snakemake.input.technoeconomic_database,
        sheet_name="gen_exist_r",
        skiprows=0,
        engine="calamine",
    )
    pd.concat([
        areas["region"],
        excel_exist_pcap_r["region"],
    ]).drop_duplicates().to_csv(
        snakemake.output.regionsdd, index=False, header=None
        )
else:
    areas["region"].drop_duplicates().to_csv(
        snakemake.output.regionsdd, index=False, header=None
    )